# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

for k, v in obj.items():
    print(f"{k:10} = '{v}'")


home_page  = 'https://github.com/pypa/sampleproject'
keywords   = 'sample setuptools development'
license    = 'MIT'


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

3. Разбейте текст формулировки задачи 2 на слова.

In [1]:
text = 'Написать регулярное выражение, которое позволит найти номера групп студентов.'
print(text.split())

['Написать', 'регулярное', 'выражение,', 'которое', 'позволит', 'найти', 'номера', 'групп', 'студентов.']


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [4]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

sample_recipes = recipes.sample(n=5)

formatted_data = [
    '|{:^12} |{:^11} |'.format(row['id'], row['minutes'])
    for _, row in sample_recipes.iterrows()
]

header = '|{:^12} |{:^11} |'.format('id', 'minutes')
horizontal_line = '|' + '-'*26 + '|'

table = header + '\n' + horizontal_line + '\n' + '\n'.join(formatted_data)

print(table)

|     id      |  minutes   |
|--------------------------|
|   66060     |    30      |
|   290219    |    20      |
|   126156    |    30      |
|   256775    |    25      |
|   434197    |    90      |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [5]:
import pandas as pd
from bs4 import BeautifulSoup

recipes = pd.read_csv('recipes_sample.csv')

with open('steps_sample.xml') as f:
    ab = BeautifulSoup(f, 'xml')

steps_dict = {int(recipe.id.text): [step.text for step in recipe.find_all('step')] for recipe in ab.find_all('recipe')}

def show_info(recipe_id):
    row = recipes.loc[recipes['id'] == recipe_id].iloc[0]
    steps = steps_dict.get(recipe_id, [])

    return f'"{row["name"].title()}"\n\n' + '\n'.join([f'{i+1}. {step.capitalize()}' for i, step in enumerate(steps)]) + \
           f'\n{"-"*10}\nАвтор: {row["contributor_id"]}\nСреднее время приготовления: {row["minutes"]} минут\n'

print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [6]:
import re
import xml.etree.ElementTree as ET

# чтение XML файла
tree = ET.parse('steps_sample.xml')
root = tree.getroot()

# поиск шагов рецепта с id 25082
for recipe in root.findall(".//recipe[id='25082']"):
    for i, step in enumerate(recipe.findall("steps/step"), 1):
        matches = re.findall(r'\d+\s+(?:hour|minute)s?', step.text)
        if matches:
            print(f"Шаг {i}: {', '.join(matches)}")



Шаг 6: 20 minutes
Шаг 8: 10 minutes
Шаг 10: 2 hours
Шаг 14: 10 minutes
Шаг 17: 20 minutes, 30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [7]:
import pandas as pd

df_recipes = pd.read_csv('recipes_sample.csv')

df_recipes['description'].fillna(' ', inplace=True)

pattern = '^this[\w\d\s]+,[ ]?but'
matching_descriptions = df_recipes[df_recipes['description'].str.contains(pattern, regex=True)]

print('Количество подходящих описаний:', matching_descriptions.shape[0])
pd.set_option('max_colwidth', int(matching_descriptions['description'].apply(len).max() + 10))
print(matching_descriptions['description'].sample(3))

Количество подходящих описаний: 134
28730                                                                    this is fabulous for a brunch, but i usually make it for dinner.
1490     this dressing is wonderful on baby greens salad, but i would put it on, or mix it in, with any salad. it's simple and delicious!
22422                     this is really great to have on hand when you have little time, but want something fresh and hot for breakfast.
Name: description, dtype: object


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [8]:
import pandas as pd
import xml.etree.ElementTree as ET
import re

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

recipe_id = '72367'
recipe_steps = root.findall(f".//recipe[id='{recipe_id}']/steps/step")

for step in recipe_steps:
    step_text = step.text
    if step_text:
        step_text = re.sub(r'\s*/\s*', '/', step_text)
        step.text = step_text

for step in recipe_steps:
    print(step.text)


mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [10]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import nltk
from nltk.tokenize.toktok import ToktokTokenizer

recipe_steps = {}
for recipe in ab.find_all('recipe'):
    recipe_id = int(recipe.id.text)
    steps = [step.text for step in recipe.find_all('step')]
    recipe_steps[recipe_id] = steps

flat_steps = [step.lower() for steps in recipe_steps.values() for step in steps]

toktok = ToktokTokenizer()

token_list = toktok.tokenize(flat_steps)
word_list = filter(str.isalpha, token_list)

unique_word_set = set(word_list)
print(f"Количество уникальных слов: {len(unique_word_set)}")


Количество уникальных слов: 14953


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [15]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [14]:
import nltk
import pandas as pd

# загрузка данных
df_recipes = pd.read_csv('recipes_sample.csv')

# проверка типов данных столбца 'description'
df_recipes = df_recipes.loc[df_recipes['description'].apply(lambda x: isinstance(x, str))]

# разбиение описаний на предложения и подсчет количества предложений
df_recipes['sentences'] = df_recipes['description'].apply(nltk.sent_tokenize)
df_recipes['sentences_count'] = df_recipes['sentences'].apply(len)

# определение 5 самых длинных описаний по количеству предложений
top_5_longest = df_recipes.nlargest(5, 'sentences_count', keep='all')

# вывод самых длинных описаний
print(top_5_longest[['name', 'sentences_count', 'description']].head())

                                                                   name  \
18408                      my favorite buttercream icing for decorating   
481           alligator claws  avocado fritters  with chipotle lime dip   
22566                                         rich barley mushroom soup   
6779                                                      chocolate tea   
16296  little bunny foo foo cake  carrot cake  with cream cheese frosti   

       sentences_count  \
18408               76   
481                 27   
22566               24   
6779                23   
16296               23   

                                                                                                                                                                                                                                                                                                                                                                                                        

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [16]:
import nltk
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

def pos_info(recipe_id):
    recipe_name = recipes.loc[recipes['id'] == recipe_id, 'name'].values[0]
    tokens = nltk.word_tokenize(recipe_name)
    pos_tags = nltk.pos_tag(tokens)
    pos_list = [pos[1] for pos in pos_tags]
    words_list = [pos[0] for pos in pos_tags]
    max_len = max(len(word) for word in words_list)
    output = ' '.join('{{:^{}}}'.format(max_len).format(pos) for pos in pos_list) + '\n'
    output += ' '.join('{{:^{}}}'.format(max_len).format(word) for word in words_list) + '\n'
    return output

print(pos_info(241106))

   JJ        NNS       IN        NNS       VBP       JJ        CC        JJ        NNS   
eggplant   steaks     with    chickpeas   feta     cheese      and      black    olives  

